In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import time

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
CREATE STAGE if not exists EXAMPLE_DOCS 
	DIRECTORY = ( ENABLE = true ) 
	ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

In [ ]:
MY_STAGE = 'EXAMPLE_DOCS/machine_reports'
MY_FILE_NAME = "data/docs/*.pdf"


# Upload the file to a stage.
put_result = session.file.put(MY_FILE_NAME, MY_STAGE, auto_compress=False,overwrite=True)
put_result[0].status

In [ ]:
SELECT AI_EXTRACT(
  file => TO_FILE('@EXAMPLE_DOCS/machine_reports','Manual_2022-02-01.pdf'),
  responseFormat => [['name', 'Who inspected the machine?'], ['date', 'What was the date of the inspection?']]
) as json_data

In [ ]:
SELECT AI_EXTRACT(
  file => TO_FILE('@EXAMPLE_DOCS/machine_reports','Manual_2022-02-01.pdf'),
  responseFormat => [['name', 'does this doc have a name? Answer Yes or No']]
) as json_data

In [ ]:
time.sleep(120)

In [ ]:
SELECT 
relative_path,
  json_data:response.date::STRING as response_date,
  json_data:response.name::STRING as inspector,
  json_data:response.grade::STRING as grade,
  json_data:response.machine::STRING as machine
from
(
SELECT 
relative_path,
AI_EXTRACT(
  file => TO_FILE('@EXAMPLE_DOCS',RELATIVE_PATH),
  responseFormat => [['name', 'Who inspected the machine?'], 
  ['date', 'What was the date of the inspection?'],
  ['grade', 'What was the grade of the inspection?'],
  ['machine', 'What machine was inspected?']]
) as json_data
from DIRECTORY(@EXAMPLE_DOCS)
where relative_path like 'machine_reports/%'
);

In [ ]:
MY_STAGE = 'EXAMPLE_DOCS/resumes'
MY_FILE_NAME = "data/docs/resumes/*.pdf"


# Upload the file to a stage.
put_result = session.file.put(MY_FILE_NAME, MY_STAGE, auto_compress=False,overwrite=True)
put_result[0].status

In [ ]:
SELECT 
  json_data:response.name::STRING as name,
  json_data:response.email::STRING as email,
  json_data:response.phone::STRING as phone_number
  
from
(
SELECT AI_EXTRACT(
  file => TO_FILE('@EXAMPLE_DOCS/resumes','resume-sample-2-13-1.pdf'),
  responseFormat => [['name', 'What is the name of the resume applicant?'], 
  ['email', 'What is the email of the resume applicant?'],
  ['phone', 'What is the phone number of the resume applicant?']]
) as json_data);

In [ ]:
SELECT 
relative_path,
SNOWFLAKE.CORTEX.PARSE_DOCUMENT('@EXAMPLE_DOCS',relative_path):content::STRING AS resume_text,
from DIRECTORY(@EXAMPLE_DOCS)
where relative_path like 'resumes/%';